# Select by Length

In [30]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {"food":"Kimchi","category":"Korean food"},
    {"food":"Chocolate","category":"dessert"},
    {"food":"Pasta","category":"Italian food"},
    {"food":"Americano","category":"Coffee"}
]

example_prompt = PromptTemplate(template="Food:{food} Category:{category}",
                                input_variables=["food","category"])                               
example_selector = LengthBasedExampleSelector(examples = examples, example_prompt=example_prompt,max_length=30)
dynamic_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "What is the category of this food?",
    suffix = "Food: {food}",
    input_variables = ["food"]
)

output=dynamic_prompt.format(food="Korean BBQ")
print(len(output.split()),output)

20 What is the category of this food?

Food:Kimchi Category:Korean food

Food:Chocolate Category:dessert

Food:Pasta Category:Italian food

Food:Americano Category:Coffee

Food: Korean BBQ


In [29]:
output=dynamic_prompt.format(food="""Korean BBQ: Grilled meat, vibrant flavors, charcoal aroma. 
Tradition meets friends and family in a delightful dining experience, creating lasting memories.
""")
print(len(output.split()),output)

34 What is the category of this food?

Food:Kimchi Category:Korean food

Food:Chocolate Category:dessert

Food: Korean BBQ: Grilled meat, vibrant flavors, charcoal aroma. 
Tradition meets friends and family in a delightful dining experience, creating lasting memories.



In [31]:
food="Korean BBQ is a traditional cuisine where fresh meat " \
"is grilled with seasonings. The aroma of burning charcoal" \
"and rich flavors from special sauces create a delightful " \
"dining experience, often shared with family or friends."
output=dynamic_prompt.format(food = food)
print(len(output.split()),output)

42 What is the category of this food?

Food: Korean BBQ is a traditional cuisine where fresh meat is grilled with seasonings. The aroma of burning charcoaland rich flavors from special sauces create a delightful dining experience, often shared with family or friends.


# N-gram Selector

In [40]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector.ngram_overlap import NGramOverlapExampleSelector

examples = [
    {"food":"Kimchi is my favorite food.","category":"Korean food"},
    {"food":"Chocolate is easy to gain weight food.","category":"dessert"},
    {"food":"I love pasta; it's my favorite comfort food after work.","category":"Italian food"},
    {"food":"Sipping an Americano, contemplating weight, a mindful morning routine begins.","category":"Coffee"}
]

example_prompt = PromptTemplate(template="Food:{food} Category:{category}",
                                input_variables=["food","category"])       
example_selector = NGramOverlapExampleSelector(examples = examples, example_prompt=example_prompt,threshold=-1.0)
dynamic_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "What is the category in the food? .",
    suffix = "Food: {food}",
    input_variables = ["food"]
)

output=dynamic_prompt.format(food="Sushi is my favorite choice for party food")
print(output)

What is the category in the food? .

Food:Kimchi is my favorite food. Category:Korean food

Food:I love pasta; it's my favorite comfort food after work. Category:Italian food

Food:Chocolate is easy to gain weight food. Category:dessert

Food:Sipping an Americano, contemplating weight, a mindful morning routine begins. Category:Coffee

Food: Sushi is my favorite choice for party food


In [41]:
example_selector = NGramOverlapExampleSelector(examples = examples, example_prompt=example_prompt,threshold=0.0)
dynamic_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "What is the category in the food? .",
    suffix = "Food: {food}",
    input_variables = ["food"]
)

output=dynamic_prompt.format(food="Sushi is my favorite choice for party food")
print(output)

What is the category in the food? .

Food:Kimchi is my favorite food. Category:Korean food

Food:I love pasta; it's my favorite comfort food after work. Category:Italian food

Food: Sushi is my favorite choice for party food


In [42]:
example_selector = NGramOverlapExampleSelector(examples = examples, example_prompt=example_prompt,threshold=-0.2)
dynamic_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "What is the category in the food? .",
    suffix = "Food: {food}",
    input_variables = ["food"]
)

output=dynamic_prompt.format(food="Sushi is my favorite choice for party food")
print(output)

What is the category in the food? .

Food:Kimchi is my favorite food. Category:Korean food

Food:I love pasta; it's my favorite comfort food after work. Category:Italian food

Food:Chocolate is easy to gain weight food. Category:dessert

Food:Sipping an Americano, contemplating weight, a mindful morning routine begins. Category:Coffee

Food: Sushi is my favorite choice for party food


# Similarity based

In [43]:
!pip install chromadb

  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/d2/24/1141557c77cf99e01baf16370ede5c0d43bef2fe17b36b8851bafd021336/chromadb-0.4.21-py3-none-any.whl.metadata
  Obtaining dependency information for chroma-hnswlib==0.7.3 from https://files.pythonhosted.org/packages/94/3f/844393b0d2ea1072b7704d6eff5c595e05ae8b831b96340cdb76b2fe995c/chroma_hnswlib-0.7.3-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for fastapi>=0.95.2 from https://files.pythonhosted.org/packages/d4/e0/d5d6482e992a1892f3a9a62f6a9154944ae5b276e7da1cf92faa02e3a107/fastapi-0.108.0-py3-none-any.whl.metadata
  Obtaining dependency information for uvicorn[standard]>=0.18.3 from https://files.pythonhosted.org/packages/26/59/fddd9df489fe27f492cc97626e03663fb3b9b6ef7ce8597a7cdc5f2cbbad/uvicorn-0.25.0-py3-none-any.whl.metadata
  Obtaining dependency information for posthog>=2.4.0 from https://files.pythonhosted.org/packages/3b/82/441cb77a43499661228048dcd0d2

In [55]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma

import os
os.environ["OPENAI_API_KEY"] = "{YOUR_OPENAI_KEY}"

examples = [
    {"input":"Happy.","category":"emotion"},
    {"input":"BBQ","category":"food"},
    {"input":"Golf","category":"Sports"},
    {"input":"Student","category":"Person"}
]

example_prompt = PromptTemplate(template="Input:{input} Category:{category}",
                                input_variables=["input","category"])       
example_selector = SemanticSimilarityExampleSelector.from_examples(examples, OpenAIEmbeddings(),Chroma,k=1,)
dynamic_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "What is the category of the input? .",
    suffix = "input: {input}",
    input_variables = ["input"]
)

output=dynamic_prompt.format(input="Sushi")
print(output)

What is the category of the input? .

Input:BBQ Category:food

input: Sushi


## MMR Selector

In [11]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain.vectorstores import Chroma

import os
os.environ["OPENAI_API_KEY"] = "{YOUR_OPENAI_KEY}"

examples = [
    {"input":"Please summarize the weather news.\n"
     ,"summary":"Today's weather: Sunny skies, mild temperatures,"\
     " and a gentle breeze. Enjoy the pleasant conditions throughout the day!"},
    {"input":"Please summarize the economy news.\n","summary":"Global stocks rise on positive economic data;"\
     "inflation concerns persist. Tech sector outperforms; central banks closely monitor."},
    {"input":"Please summarize retail news.\n","summary":"Major retailer announces record-breaking sales during holiday shopping season"},
    {"input":"What is stock market trend?\n","summary":"Investor optimism grows amid easing global trade tensions"},
    {"input":"Typhoon related news.\n","summary":"IAirports and schools close ahead of approaching typhoon threat"},
    
]

example_prompt = PromptTemplate(template="Input:{input} Summary:{summary}",
                                input_variables=["input","summary"])       
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(examples, OpenAIEmbeddings(),Chroma,k=2,)
dynamic_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    suffix = "input: {input}\nSummary:",
    prefix = "",
    input_variables = ["input"]
)

output=dynamic_prompt.format(input="I want to know the economy trends and weather this week.")
print(output)

Input:Please summarize the weather news.
 Summary:Today's weather: Sunny skies, mild temperatures, and a gentle breeze. Enjoy the pleasant conditions throughout the day!

Input:What is stock market trend? Summary:Investor optimism grows amid easing global trade tensions

input: I want to know the economy trends and weather this week.
Summary:
